# Optimización de Hiperparámetros en Redes Neuronales

En este notebook abordaremos el tema de la Optimización de Hiperparámetros (HPO, por sus siglas en inglés: **Hyperparameter Optimization**) en redes neuronales. Para ello:

1. Explicaremos qué es la optimización de hiperparámetros en detalle.
2. Presentaremos un problema de HPO con una red neuronal tipo MLP sobre el dataset MNIST utilizando PyTorch.
3. Desarrollaremos un ejemplo utilizando **Grid Search** (búsqueda en rejilla), explicando su funcionamiento y sus ventajas y desventajas.
4. Realizaremos otro ejemplo con **Random Search** (búsqueda aleatoria).
5. Presentaremos el concepto de **Optimización Bayesiana** y mostraremos un ejemplo práctico aplicando esta técnica al problema planteado.

---

## 1. ¿Qué es la Optimización de Hiperparámetros?

Cuando entrenamos una red neuronal, no sólo debemos preocuparnos por los parámetros aprendibles del modelo (los pesos y sesgos), sino también por aquellos parámetros que no se aprenden directamente a partir de los datos y deben definirse antes del entrenamiento: los **hiperparámetros**. Ejemplos de hiperparámetros incluyen:

- La tasa de aprendizaje (learning rate) del optimizador.
- El número de épocas.
- Relacionados con la arquitectura de la red (número de capas, número de neuronas por capa).
- El tamaño de batch.
- Parámetros de regularización (dropout, L2, etc.).

La optimización de hiperparámetros consiste en encontrar la combinación de estos que maximice el rendimiento del modelo en un conjunto de validación o a través de alguna métrica establecida.

Matemáticamente, si consideramos un conjunto de hiperparámetros $ \lambda = (\lambda_1, \lambda_2, ..., \lambda_k) $, queremos resolver:

$$
\lambda^* = \underset{\lambda \in \Lambda}{\mathrm{argmax}}\; f(\lambda)
$$

donde $ f(\lambda) $ es la función objetivo (típicamente el rendimiento del modelo en un conjunto de validación) y $ \Lambda $ es el espacio de búsqueda de hiperparámetros.

---

## 2. Problema de Ejemplo: MLP en MNIST con PyTorch

Utilizaremos el dataset MNIST, un conjunto de datos muy conocido que contiene imágenes de dígitos escritos a mano (0 a 9). Entrenaremos una red neuronal MLP (Perceptrón Multicapa) sobre este dataset y buscaremos optimizar algunos hiperparámetros como:

- La tasa de aprendizaje (learning rate)
- El tamaño de batch (batch size)
- El número de neuronas en la capa oculta

Primero, construiremos la infraestructura básica: cargar el dataset, definir la red, la función de entrenamiento y validación.

In [1]:
# Importaciones y configuración

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Para asegurarnos reproducibilidad
torch.manual_seed(0)

# Transformaciones de imágenes: normalización a 0.1307 y desviación estándar 0.3081 (valores estándar para MNIST)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Cargar datasets de entrenamiento y prueba
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Función que construye un MLP dado un número de neuronas en capa oculta y número de clases
class MLP(nn.Module):
    def __init__(self, input_size=784, hidden_size=128, num_classes=10):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        x = x.view(x.size(0), -1)
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

def train_one_epoch(model, optimizer, criterion, dataloader, device='cpu'):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
    return running_loss / total, 100. * correct / total

def evaluate(model, criterion, dataloader, device='cpu'):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
    return running_loss / total, 100. * correct / total

## 3. Optimización de Hiperparámetros con Grid Search

La **Búsqueda en Rejilla (Grid Search)** consiste en definir un conjunto discreto y finito de valores posibles para cada hiperparámetro y luego entrenar el modelo para todas las combinaciones posibles. Al final, seleccionamos la combinación de hiperparámetros que mejor rendimiento haya obtenido.

Por ejemplo, si queremos buscar sobre:

- Learning rate: $\{0.01, 0.001\}$
- Hidden size: $\{64, 128\}$
- Batch size: $\{64, 128\}$

Tendremos $2 \times 2 \times 2 = 8$ combinaciones a evaluar.

La búsqueda en rejilla es simple de implementar y fácil de paralelizar, pero el costo computacional crece exponencialmente con el número de hiperparámetros y el número de valores por hiperparámetro. Además, no aprovecha información de corridas previas, simplemente explora todas las combinaciones.

A continuación, mostraremos un ejemplo.

In [2]:
# Ejemplo de Grid Search

from itertools import product

device = 'cuda' if torch.cuda.is_available() else 'cpu'

param_grid = {
    'lr': [0.1, 0.01],
    'hidden_size': [64, 128],
    'batch_size': [64, 128]
}

epochs = 5

best_acc = 0.0
best_params = None

for lr, hidden_size, batch_size in product(param_grid['lr'], param_grid['hidden_size'], param_grid['batch_size']):
    # Cargar datos con el batch_size actual
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    model = MLP(hidden_size=hidden_size).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr)
    
    for ep in range(epochs):
        train_loss, train_acc = train_one_epoch(model, optimizer, criterion, train_loader, device)
        val_loss, val_acc = evaluate(model, criterion, val_loader, device)

    print(f'Resultados para lr={lr}, hidden_size={hidden_size}, batch_size={batch_size}:')
    print(f'\tExactitud de entrenamiento: {train_acc:.2f}%')
    print(f'\tExactitud de validación: {val_acc:.2f}%')
        
    if val_acc > best_acc:
        best_acc = val_acc
        best_params = {'lr': lr, 'hidden_size': hidden_size, 'batch_size': batch_size}

print("Mejores parámetros encontrados con Grid Search:", best_params, "con exactitud:", best_acc)

Resultados para lr=0.1, hidden_size=64, batch_size=64:
	Exactitud de entrenamiento: 98.11%
	Exactitud de validación: 97.37%
Resultados para lr=0.1, hidden_size=64, batch_size=128:
	Exactitud de entrenamiento: 97.58%
	Exactitud de validación: 97.16%
Resultados para lr=0.1, hidden_size=128, batch_size=64:
	Exactitud de entrenamiento: 98.59%
	Exactitud de validación: 97.67%
Resultados para lr=0.1, hidden_size=128, batch_size=128:
	Exactitud de entrenamiento: 97.85%
	Exactitud de validación: 97.15%
Resultados para lr=0.01, hidden_size=64, batch_size=64:
	Exactitud de entrenamiento: 93.94%
	Exactitud de validación: 94.24%
Resultados para lr=0.01, hidden_size=64, batch_size=128:
	Exactitud de entrenamiento: 92.47%
	Exactitud de validación: 93.11%
Resultados para lr=0.01, hidden_size=128, batch_size=64:
	Exactitud de entrenamiento: 94.21%
	Exactitud de validación: 94.64%
Resultados para lr=0.01, hidden_size=128, batch_size=128:
	Exactitud de entrenamiento: 92.55%
	Exactitud de validación: 93.

## 4. Optimización de Hiperparámetros con Random Search

**Random Search** (Búsqueda Aleatoria) selecciona combinaciones aleatorias de hiperparámetros a partir de distribuciones definidas. A diferencia de la rejilla, no se explora exhaustivamente todo el espacio. Esto puede ser ventajoso cuando el espacio es muy grande, ya que se ha demostrado que la búsqueda aleatoria puede ser más eficiente que la rejilla en muchos casos, sobre todo cuando sólo unos pocos hiperparámetros influyen significativamente en el resultado.

En términos matemáticos, si $\lambda$ es un vector de hiperparámetros, la Búsqueda Aleatoria extrae $\lambda$ de una distribución $ p(\lambda) $ (por ejemplo, uniforme).

$$
\lambda \sim p(\lambda)
$$

Luego se evalúa el rendimiento y se guarda el mejor resultado.

En este ejemplo vamos a usar listas con valores definidos a partir de las cuales escogemos valores al azar. Algo más complejo sería definir valores mínimo y máximo y escoger valores al azar entre medias, con una distribución uniforme, normal centrada en la media, log-normal para algunos hiperparámetros como la tasa de aprendizaje, etcétera.

In [3]:
import random

# Definir las distribuciones (podemos usar distribuciones simples)
# Por ejemplo, lo más simple que podemos hacer es definir listas con posibles valores: 
lr_candidates = [0.1, 0.01, 0.001, 0.0001]
hidden_size_candidates = [64, 128, 256]
batch_size_candidates = [32, 64, 128]

num_samples = 5  # Numero de combinaciones aleatorias a probar
epochs = 5

best_acc_random = 0.0
best_params_random = None

for _ in range(num_samples):
    lr = random.choice(lr_candidates)
    hidden_size = random.choice(hidden_size_candidates)
    batch_size = random.choice(batch_size_candidates)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    model = MLP(hidden_size=hidden_size).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr)
    
    for ep in range(epochs):
        train_loss, train_acc = train_one_epoch(model, optimizer, criterion, train_loader, device)
        val_loss, val_acc = evaluate(model, criterion, val_loader, device)

    print(f'Resultados para lr={lr}, hidden_size={hidden_size}, batch_size={batch_size}:')
    print(f'\tExactitud de entrenamiento: {train_acc:.2f}%')
    print(f'\tExactitud de validación: {val_acc:.2f}%')
        
    if val_acc > best_acc_random:
        best_acc_random = val_acc
        best_params_random = {'lr': lr, 'hidden_size': hidden_size, 'batch_size': batch_size}

print("Mejores parámetros encontrados con Random Search:", best_params_random, "con exactitud:", best_acc_random)

Resultados para lr=0.01, hidden_size=256, batch_size=64:
	Exactitud de entrenamiento: 94.62%
	Exactitud de validación: 94.82%
Resultados para lr=0.001, hidden_size=128, batch_size=128:
	Exactitud de entrenamiento: 85.89%
	Exactitud de validación: 87.21%
Resultados para lr=0.001, hidden_size=64, batch_size=64:
	Exactitud de entrenamiento: 88.33%
	Exactitud de validación: 89.11%
Resultados para lr=0.0001, hidden_size=128, batch_size=32:
	Exactitud de entrenamiento: 79.04%
	Exactitud de validación: 80.68%
Resultados para lr=0.001, hidden_size=64, batch_size=128:
	Exactitud de entrenamiento: 85.78%
	Exactitud de validación: 87.05%
Mejores parámetros encontrados con Random Search: {'lr': 0.01, 'hidden_size': 256, 'batch_size': 64} con exactitud: 94.82


## 5. Optimización Bayesiana

La **Optimización Bayesiana** es un enfoque más sofisticado para la búsqueda de hiperparámetros. A diferencia de Grid Search o Random Search, la optimización bayesiana utiliza información de las evaluaciones previas del modelo para guiar la búsqueda hacia regiones más prometedoras del espacio de hiperparámetros.

La idea principal es modelar la función objetivo $ f(\lambda) $ a través de un modelo probabilístico (por ejemplo, un Proceso Gaussiano). Conforme se evalúan distintos hiperparámetros, este modelo probabilístico se actualiza, mejorando su aproximación de $ f(\lambda) $. Luego se aplica una **función de adquisición** que sugiere el siguiente punto (conjunto de hiperparámetros) a evaluar, balanceando la exploración y la explotación del espacio.

Matemáticamente, el proceso se puede describir como:

1. Asumimos un prior sobre la función objetivo $ f(\lambda) $, típicamente un proceso gaussiano $ f(\lambda) \sim GP(m(\lambda), k(\lambda,\lambda')) $.
2. Evaluamos la función en unos pocos puntos y actualizamos el posterior del GP.
3. Elegimos el próximo punto a evaluar maximizando una función de adquisición $ a(\lambda) $ (por ejemplo: Expected Improvement, UCB, etc.):
$$
\lambda_{n+1} = \underset{\lambda \in \Lambda}{\mathrm{argmax}} \; a(\lambda)
$$
4. Repetimos el proceso hasta converger o agotar recursos.

Para este ejemplo utilizaremos la librería `hyperopt` para implementar una búsqueda bayesiana sencilla.

In [4]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK

# Función objetivo que recibe un diccionario con los hiperparámetros y devuelve la métrica y un status OK, necesario para hyperopt
def objective(params):
    lr = params['lr']
    hidden_size = int(params['hidden_size'])
    batch_size = int(params['batch_size'])
    epochs = 5

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    model = MLP(hidden_size=hidden_size).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr)
    
    for ep in range(epochs):
        train_loss, train_acc = train_one_epoch(model, optimizer, criterion, train_loader, device)
        val_loss, val_acc = evaluate(model, criterion, val_loader, device)

    print(f'Resultados para lr={lr}, hidden_size={hidden_size}, batch_size={batch_size}:')
    print(f'\tExactitud de entrenamiento: {train_acc:.2f}%')
    print(f'\tExactitud de validación: {val_acc:.2f}%')

    # Queremos maximizar val_acc, pero hyperopt minimiza. Usamos -val_acc.
    return {'loss': -val_acc, 'status': STATUS_OK}

# Espacio de búsqueda
space = {
    'lr': hp.loguniform('lr', -5, 0),          # entre ~0.0067 y 1.0
    'hidden_size': hp.choice('hidden_size', [64, 128, 256]),
    'batch_size': hp.choice('batch_size', [32, 64, 128])
}

# Trials: Para guardar los resultados de cada iteración
trials = Trials()

# tpe: Tree of Parzen Estimators, una librería de optimización bayesiana
# algo: suggest, un algoritmo que sugiere los siguientes hiperparámetros a probar
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=10, trials=trials)
print("Mejores hiperparámetros encontrados (Bayesiano):", best)

Resultados para lr=0.5566060853839194, hidden_size=128, batch_size=32:
	Exactitud de entrenamiento: 93.44%                   
	Exactitud de validación: 92.96%                      
Resultados para lr=0.023028410254015495, hidden_size=64, batch_size=64:
	Exactitud de entrenamiento: 95.98%                                 
	Exactitud de validación: 95.90%                                    
Resultados para lr=0.036661069573523226, hidden_size=128, batch_size=128:
	Exactitud de entrenamiento: 96.03%                                
	Exactitud de validación: 96.25%                                   
Resultados para lr=0.12833572792811745, hidden_size=128, batch_size=64:
	Exactitud de entrenamiento: 98.76%                                 
	Exactitud de validación: 97.41%                                    
Resultados para lr=0.153444193767084, hidden_size=64, batch_size=64:
	Exactitud de entrenamiento: 98.26%                                 
	Exactitud de validación: 97.28%                   

## Conclusión

Hemos visto diferentes enfoques para la optimización de hiperparámetros:

- **Grid Search**: Exhaustivo y fácil, pero caro computacionalmente y no escalable.
- **Random Search**: Más eficiente y escalable que Grid Search, puede encontrar buenas soluciones más rápido.
- **Optimización Bayesiana**: Utiliza información de las evaluaciones previas para guiar la búsqueda, suele ser más eficiente en problemas complejos.

Dependiendo de los recursos disponibles y la complejidad del problema, se puede elegir una u otra estrategia. En problemas del mundo real con muchos hiperparámetros, la optimización bayesiana es a menudo la más eficiente.